## --- Day 5: Hydrothermal Venture ---
You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2
Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.
For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:

.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....
In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?

Your puzzle answer was 7644.

## --- Part Two ---
Unfortunately, considering only horizontal and vertical lines doesn't give you the full picture; you need to also consider diagonal lines.

Because of the limits of the hydrothermal vent mapping system, the lines in your list will only ever be horizontal, vertical, or a diagonal line at exactly 45 degrees. In other words:

An entry like 1,1 -> 3,3 covers points 1,1, 2,2, and 3,3.
An entry like 9,7 -> 7,9 covers points 9,7, 8,8, and 7,9.
Considering all lines from the above example would now produce the following diagram:

1.1....11.
.111...2..
..2.1.111.
...1.2.2..
.112313211
...1.2....
..1...1...
.1.....1..
1.......1.
222111....
You still need to determine the number of points where at least two lines overlap. In the above example, this is still anywhere in the diagram with a 2 or larger - now a total of 12 points.

Consider all of the lines. At how many points do at least two lines overlap?

Your puzzle answer was 18627.

Both parts of this puzzle are complete! They provide two gold stars: **

In [1]:
import numpy as np

def read_file(filename):
    # read file
    with open(filename) as f:
        lines = f.readlines()
    output = [str.rstrip(line) for line in lines]    
    return output

class Point(object):
    '''Creates a point on a coordinate plane with from a list [x,y].'''

    def __init__(self, coord):
        '''Defines x and y variables'''
        self.x = int(coord[0])
        self.y = int(coord[1])
    
    def display(self):
        print(self.x, self.y)
        
def is_horizontal_or_vertical(line):
    return (line.start.x == line.stop.x) | (line.start.y == line.stop.y)

In [36]:
def sorted_coords(start, stop):
    if start > stop:
        stop = stop - 1
        increment = -1
    else:
        stop = stop + 1
        increment = 1
    return np.arange(start, stop, increment)

def range_x(line):
    return sorted_coords(line.start.x,line.stop.x)

def range_y(line):
    return sorted_coords(line.start.y,line.stop.y)

def array_to_points(val):
    val = list(val)
    return [Point(coord) for coord in val]

class Line(object):
    
    def __init__(self, point):
        self.start = point[0]
        self.stop = point[1]
    
    def display(self):
        print('start x,y:', self.start.x, self.start.y)
        print('stop x,y:', self.stop.x, self.stop.y)
        
    def max_x(self):
        return max(self.start.x, self.stop.x)
    
    def max_y(self):
        return max(self.start.y, self.stop.y)
    
    def all_points(self):
        if (self.stop.x - self.start.x) == 0:
            y = range_y(self)
            x = np.zeros_like(y) + self.stop.x
        else:
            m = (self.stop.y - self.start.y) / (self.stop.x - self.start.x)
            b = self.stop.y - (m*self.stop.x)
            x = range_x(self)
            y = m*x + b
        return array_to_points(np.stack([y,x]).T.astype('int'))


In [37]:
data = read_file('./day5.txt')
data = [val.split(' -> ') for val in data]
coordinates = [[Point(val[0].split(',')),Point(val[1].split(','))]  for val in data]
lines = [Line(val) for val in coordinates]

In [38]:
part1_lines = [val for val in lines if is_horizontal_or_vertical(val)]

In [39]:
x_max = max([val.max_x() for val in lines])
y_max = max([val.max_y() for val in lines])

In [42]:
part1 = False
if part1:
    all_lines = part1_lines
else:
    all_lines = lines
    
mask = np.zeros([x_max+1,y_max+1])

for line in all_lines:
    out = line.all_points()
    for val in out:
        mask[val.x, val.y] = mask[val.x, val.y] + 1
        
print(np.sum((mask>1) + 0))

18627
